### Notes
 Here, I want to take a stab at scraping and organizing the DOR website

In [100]:
Website: 
* https://www.dor.ca.gov/

Loose inspirations: 
* Perplexity 
* Glean 
* Diffbot 

Stricter inspirations: 
* https://llamahub.ai/l/web-sitemap?from=all
* https://llamahub.ai/l/web-whole_site?from=all

https://blog.llamaindex.ai/ragarch-building-a-no-code-rag-pipeline-configuration-one-click-rag-code-generation-tool-powered-b6e8eeb70089
https://christophergs.com/blog/ai-engineering-retrieval-augmented-generation-rag-llama-index
https://github.com/run-llama/llamabot

structured hierarchical retriever 
https://docs.llamaindex.ai/en/stable/examples/query_engine/multi_doc_auto_retrieval/multi_doc_auto_retrieval.html
https://medium.com/enterprise-rag/injecting-knowledge-graphs-in-different-rag-stages-a3cd1221f57b

https://github.com/cpacker/MemGPT

contextchatengine

https://jasonwebster.dev/blog/chatting-to-a-website-with-langchain-openai-and-chromadb


SyntaxError: invalid syntax (2547230184.py, line 1)

In [ ]:
https://www.anyscale.com/blog/a-comprehensive-guide-for-building-rag-based-llm-applications-part-1

In [ ]:
# build a dataset of 

# paths of all pages on the website 

# robots.txt on cal.gov allwos scraping! great! 

# step 1) Run web spider over DoR.gov 


# maybe try vectara
https://github.com/vectara/example-notebooks/blob/main/notebooks/using-vectara-with-llamaindex.ipynb


In [ ]:
# run the spider over the website a bit? 


In [ ]:
path = './RAD15/paths.json'

In [ ]:
import json

In [ ]:
path_records = json.loads(open(path,'r').read())

In [ ]:
import pandas as pd

/var/folders/m9/_kdm7w890gl0g2x2wc8ys_880000gn/T/ipykernel_4918/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
urls_df = pd.DataFrame.from_records(path_records)[['parent_url','child_url']]

In [ ]:
# now I have the urls df

In [ ]:
urls_df['parent_url'] = urls_df.parent_url.str.replace('www.','')

In [ ]:
urls_df[['parent_url','child_url']].drop_duplicates(inplace = True)

In [ ]:
urls_df['parent_is_archive'] = urls_df['parent_url'].str.contains('MeetingArchive')

In [ ]:
nonarchive_urls_df = urls_df[~urls_df['parent_is_archive']][['parent_url','child_url']]

In [ ]:
remove_words = ['Solicitations','Meeting', 'vrcintro','VRCIntro','VrcIntro','Archive']
for r in remove_words:
    nonarchive_urls_df = nonarchive_urls_df[~nonarchive_urls_df['parent_url'].str.contains(r)]

In [ ]:
nonarchive_urls_df['parent_url'].value_counts()

parent_url
https://dor.ca.gov/Home/SsDistrictBios                162
https://dor.ca.gov/Home/Publications                   94
https://dor.ca.gov/Home/Forms                          80
https://dor.ca.gov/home/UpdatesForCRP                  72
https://dor.ca.gov/Home/VoiceOptions                   52
                                                     ... 
https://dor.ca.gov/Home/PublicForums                    1
https://dor.ca.gov/Home/FindAJob                        1
https://dor.ca.gov/Home/JobResources                    1
https://dor.ca.gov/Home/SupportedEmploymentProgram      1
https://dor.ca.gov/Home/ContactUs                       1
Name: count, Length: 318, dtype: int64

In [ ]:
atsd_df = nonarchive_urls_df[nonarchive_urls_df['parent_url'].str.contains('atsd')]
nonarchive_urls_df = nonarchive_urls_df[~nonarchive_urls_df['parent_url'].str.contains('atsd')]

In [ ]:
nonarchive_urls_df

,parent_url,child_url
0,https://dor.ca.gov/,https://publicaccessstorage.blob.core.usgovclo...
1,https://dor.ca.gov/,https://www.dor.ca.gov/
2,https://dor.ca.gov/Home/disasterpreparedness,https://www.redcross.org/content/dam/redcross/...
3,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...
4,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...
...,...,...
5927,https://dor.ca.gov/Home/Bac,https://publicaccessstorage.blob.core.usgovclo...
5928,https://dor.ca.gov/Home/BacMembership,https://www.dor.ca.gov/Home/Bac
5929,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...
5930,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...


In [ ]:
nonarchive_urls_df['last_slash'] = nonarchive_urls_df['parent_url'].str.split('/').str[-1]

In [ ]:
# figure out if something is a city or county in california 

In [ ]:
import instructor
from openai import OpenAI
from pydantic import BaseModel

# Enables `response_model`
client = instructor.patch(OpenAI())


In [146]:
from pydantic import BaseModel
from typing import List

from llama_index.program import OpenAIPydanticProgram

In [148]:
from enum import Enum

class BooleanAnswer(BaseModel): 
    '''
    Data model for a boolean answer
    '''
    explanation: str
    final_answer: bool 


In [157]:
prompt_template_str = """
The final url of this website shows {last} at the end of it. 
Based only on this, please tell me whether this is definitely the name for a city or county in California. Please assess whether this is true or not and explain why. Only answer true if you are confident. Provide the answer in a structured json."
"""
program = OpenAIPydanticProgram.from_defaults(
    output_cls=BooleanAnswer,
    llm=OpenAI(model="gpt-3.5-turbo-1106"),
    prompt_template_str=prompt_template_str,
    allow_multiple=False,
    verbose=True,
)

In [155]:
annotate_df = nonarchive_urls_df[['parent_url','last_slash']].drop_duplicates()

In [161]:
records = []
for _, row in annotate_df.iterrows(): 
    if not (row.last_slash == ''):
        result = program(last = row.last_slash)
        records.append({'parent_url' : row.parent_url, 'last_slash': row.last_slash, 'answer':result.final_answer, 'explanation': result.explanation})
        print(records[-1])
        print('\n\n')

Function call: BooleanAnswer with args: {"explanation":"The presence of 'disasterpreparedness' in the final URL does not necessarily indicate that it is the name of a city or county in California. It could be a website dedicated to disaster preparedness in California, but it does not confirm that 'disasterpreparedness' is the name of a city or county. Therefore, I am not confident that this is the name for a city or county in California.","final_answer":false}
{'parent_url': 'https://dor.ca.gov/Home/disasterpreparedness', 'last_slash': 'disasterpreparedness', 'answer': False, 'explanation': "The presence of 'disasterpreparedness' in the final URL does not necessarily indicate that it is the name of a city or county in California. It could be a website dedicated to disaster preparedness in California, but it does not confirm that 'disasterpreparedness' is the name of a city or county. Therefore, I am not confident that this is the name for a city or county in California."}



Function c

In [162]:
import pickle

In [165]:
city_county_df = pd.DataFrame(records)

In [176]:
nonarchive_urls_df = nonarchive_urls_df.merge(city_county_df[['parent_url','answer']], how = 'left')

In [178]:
nonarchive_urls_df.query('answer == True')

,parent_url,child_url,last_slash,answer
945,https://dor.ca.gov/Home/Pasadena,https://publicaccessstorage.blob.core.usgovclo...,Pasadena,True
946,https://dor.ca.gov/Home/Pasadena,https://publicaccessstorage.blob.core.usgovclo...,Pasadena,True
947,https://dor.ca.gov/Home/Pasadena,https://publicaccessstorage.blob.core.usgovclo...,Pasadena,True
948,https://dor.ca.gov/Home/Pasadena,https://publicaccessstorage.blob.core.usgovclo...,Pasadena,True
950,https://dor.ca.gov/Home/SantaClarita,https://publicaccessstorage.blob.core.usgovclo...,SantaClarita,True
...,...,...,...,...
2992,https://dor.ca.gov/Home/Fremont,https://publicaccessstorage.blob.core.usgovclo...,Fremont,True
2993,https://dor.ca.gov/Home/Fremont,https://publicaccessstorage.blob.core.usgovclo...,Fremont,True
2994,https://dor.ca.gov/Home/Fremont,https://publicaccessstorage.blob.core.usgovclo...,Fremont,True
2995,https://dor.ca.gov/Home/Fremont,https://publicaccessstorage.blob.core.usgovclo...,Fremont,True


In [186]:
nonarchive_urls_df

,parent_url,child_url,last_slash,answer
0,https://dor.ca.gov/,https://publicaccessstorage.blob.core.usgovclo...,,NaN
1,https://dor.ca.gov/,https://www.dor.ca.gov/,,NaN
2,https://dor.ca.gov/Home/disasterpreparedness,https://www.redcross.org/content/dam/redcross/...,disasterpreparedness,False
3,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False
4,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False
...,...,...,...,...
3001,https://dor.ca.gov/Home/Bac,https://publicaccessstorage.blob.core.usgovclo...,Bac,False
3002,https://dor.ca.gov/Home/BacMembership,https://www.dor.ca.gov/Home/Bac,BacMembership,False
3003,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False
3004,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False


In [193]:
nonarchive_urls_df['child_is_file'] = nonarchive_urls_df['child_url'].str.split('/').str[-1].str.contains('.')
nonarchive_urls_df['child_is_pdf'] = nonarchive_urls_df['child_url'].str.contains('.pdf')
nonarchive_urls_df['is_county_or_city']  = nonarchive_urls_df['answer'] == True

In [207]:
noncity_urls_df = nonarchive_urls_df[~nonarchive_urls_df['is_county_or_city']]

In [195]:
pdf_url_df = nonarchive_urls_df.query('child_is_pdf == True')

In [236]:
final_url_list = [ur for ur in set(nonarchive_urls_df['parent_url'].tolist()) if not '.' in ur.split('/')[-1]]

In [237]:
len(final_url_list)

316

In [220]:
pdf_url_df.query('is_county_or_city == False')

,parent_url,child_url,last_slash,answer,child_is_file,child_is_pdf,is_county_or_city
2,https://dor.ca.gov/Home/disasterpreparedness,https://www.redcross.org/content/dam/redcross/...,disasterpreparedness,False,True,True,False
3,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
4,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
5,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
6,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
...,...,...,...,...,...,...,...
2983,https://dor.ca.gov/Home/Antioch,https://publicaccessstorage.blob.core.usgovclo...,Antioch,False,True,True,False
2984,https://dor.ca.gov/Home/Antioch,https://publicaccessstorage.blob.core.usgovclo...,Antioch,False,True,True,False
3003,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False,True,True,False
3004,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False,True,True,False


In [238]:
pickle.dump(final_url_list, open('final_url_list.pkl','wb'))

In [222]:
nonarchive_urls_df.to_pickle('nonarchive_urls_df')

In [223]:
pdf_url_df.to_pickle('pdf_url_df.pkl')

In [249]:
web_crawl_json = json.loads(open('crawl.json').read()) + json.loads(open('crawl2.json').read()) + json.loads(open('crawl3.json').read())

In [250]:
fus = set([f.replace('www.','') for f in final_url_list])

In [251]:
wus = set([w['url'].replace('www.','') for w in web_crawl_json])

In [252]:
fus-wus

set()

In [255]:
pdf_url_df['child_url'].tolist()

['https://www.redcross.org/content/dam/redcross/atg/PDF_s/Preparedness___Disaster_Recovery/General_Preparedness___Recovery/Home/A4497.pdf',
 'https://publicaccessstorage.blob.core.usgovcloudapi.net/publicsitefiles/DOR%20Documents/Disaster%20Preparedness/Cal%20OESDOR%20-%20AFN%20Disaster%20Preparedness%20Guide_Armenian.pdf',
 'https://publicaccessstorage.blob.core.usgovcloudapi.net/publicsitefiles/DOR%20Documents/Disaster%20Preparedness/Cal%20OESDOR%20-%20AFN%20Disaster%20Preparedness%20Guide_Hmong.pdf',
 'https://publicaccessstorage.blob.core.usgovcloudapi.net/publicsitefiles/DOR%20Documents/Disaster%20Preparedness/Cal%20OESDOR%20-%20AFN%20Disaster%20Preparedness%20Guide_Korean.pdf',
 'https://publicaccessstorage.blob.core.usgovcloudapi.net/publicsitefiles/DOR%20Documents/Disaster%20Preparedness/Cal%20OESDOR%20-%20AFN%20Disaster%20Preparedness%20Guide_Russian.pdf',
 'https://publicaccessstorage.blob.core.usgovcloudapi.net/publicsitefiles/DOR%20Documents/Disaster%20Preparedness/Cal%20OE

In [244]:
json.dump(list(fus-wus), open('newcrawl.txt','w'))

In [ ]:
# 253 parent webpages 
# discounting archive meetings 
# discounting counties and cities 

In [ ]:
# now grab all the html pages for these 

In [180]:
# 548 webpages are cities or counties in california 

In [ ]:
url='https://dor.ca.gov/Home/SanLuisObispo' url_end='SanLuisObispo' is_count_or_city=<Option.maybe: 'maybe'> which_state='California'

In [159]:
program(last = 'SanFrancisco')

Function call: BooleanAnswer with args: {"explanation":"The presence of 'SanFrancisco' at the end of the URL indicates that it is likely the name of a city in California. San Francisco is a well-known city in California, and it is common for city names to appear in URLs. Therefore, it is highly likely that 'SanFrancisco' refers to the name of a city in California.","final_answer":true}


BooleanAnswer(explanation="The presence of 'SanFrancisco' at the end of the URL indicates that it is likely the name of a city in California. San Francisco is a well-known city in California, and it is common for city names to appear in URLs. Therefore, it is highly likely that 'SanFrancisco' refers to the name of a city in California.", final_answer=True)

In [160]:
program(last = 'SanLuisObispo')

Function call: BooleanAnswer with args: {"explanation":"San Luis Obispo is the name of a city in California. It is the county seat of San Luis Obispo County. Therefore, based on the information provided, it is definitely the name of a city or county in California.","final_answer":true}


BooleanAnswer(explanation='San Luis Obispo is the name of a city in California. It is the county seat of San Luis Obispo County. Therefore, based on the information provided, it is definitely the name of a city or county in California.', final_answer=True)

IndividualServiceProviders
ProposedRulemakingAndRegs


In [ ]:
# what is this document? 
# who is it for? 
# how do you use it? 

In [ ]:
separate_these = ['SsDistrictBios']
remove_these = ['vrcintro','VRCIntro','VrcIntro']

In [ ]:
# just do pdfs 
# and the markdown rendering of the pages 

In [ ]:
from IPython.display import HTML 

In [ ]:
html = urls_df.to_html()

### okay now we need to save the unified data somewhere accessible 

In [257]:
json.dump(web_crawl_json, open('webcrawl.json','w'))

In [258]:
# we have all the parents
# we have pdf children 


,parent_url,child_url,last_slash,answer,child_is_file,child_is_pdf,is_county_or_city
0,https://dor.ca.gov/,https://publicaccessstorage.blob.core.usgovclo...,,NaN,True,False,False
1,https://dor.ca.gov/,https://www.dor.ca.gov/,,NaN,False,False,False
2,https://dor.ca.gov/Home/disasterpreparedness,https://www.redcross.org/content/dam/redcross/...,disasterpreparedness,False,True,True,False
3,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
4,https://dor.ca.gov/Home/disasterpreparedness,https://publicaccessstorage.blob.core.usgovclo...,disasterpreparedness,False,True,True,False
...,...,...,...,...,...,...,...
3001,https://dor.ca.gov/Home/Bac,https://publicaccessstorage.blob.core.usgovclo...,Bac,False,True,False,False
3002,https://dor.ca.gov/Home/BacMembership,https://www.dor.ca.gov/Home/Bac,BacMembership,False,True,False,False
3003,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False,True,True,False
3004,https://dor.ca.gov/Home/Atac,https://publicaccessstorage.blob.core.usgovclo...,Atac,False,True,True,False


In [261]:
import glob

In [264]:
local_pdfs_df = pd.DataFrame({'path':glob.glob('./pdfs/*')})

In [269]:
local_pdfs_df['filename'] = local_pdfs_df.path.str.replace('./pdfs/','')

In [273]:
pdf_url_df['filename'] = pdf_url_df['child_url'].str.split('/').str[-1]

/var/folders/m9/_kdm7w890gl0g2x2wc8ys_880000gn/T/ipykernel_4918/547863632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf_url_df['filename'] = pdf_url_df['child_url'].str.split('/').str[-1]


In [276]:
set(pdf_url_df['filename'].tolist()) - set(local_pdfs_df['filename'].tolist())

{'4833.pdf'}

In [283]:
final_pdf_url_df = pdf_url_df.merge(local_pdfs_df, how = 'inner', on = 'filename')

In [285]:
final_pdf_url_df.to_pickle('final_pdf_url_df.pkl')

In [288]:
webpage_df = nonarchive_urls_df[['parent_url']].drop_duplicates()

In [291]:
web_crawl_df = pd.DataFrame(web_crawl_json)

In [295]:
final_parent_url_df = webpage_df.merge(web_crawl_df, how = 'inner', left_on = 'parent_url', right_on = 'url')

In [297]:
final_parent_url_df.to_pickle('final_parent_url_df.pkl')

In [296]:
nonarchive_urls_df.to_pickle('nonarchive_urls_df.pkl')

In [300]:
final_webpage_df = webpage_df.merge(web_crawl_df, how = 'inner', left_on = 'parent_url', right_on = 'url')

In [301]:
final_webpage_df.to_pickle('final_webpage_df.pkl')

the final dataframes are 
* final_parent_url_df
* final_webpage_df

also requires 
* ./pdfs/*.pdf